In [1]:
import pandas as pd
from SerialLogger import load_yaml_header
from Analyzelogs import load_log, categorize_logfiles, calculate_all, calculate_cost, extract_avg

# import all used functions for processing the data

In [ ]:
# categorize device name here, do multiple if multiple devices.
# 


root_dir = "./test_results" # TODO set to your workspace root or use relative path
device_dir = "your_device_folder" # TODO set to the folder containing logfiles

device_dict = categorize_logfiles(f"{root_dir}/{device_dir})

In [ ]:
# create index, the full name of each primitive being benchmarked
index = ["XOR" , "RNG"] #TODO fill in with your primitives

# create corresponding list to the index that contains the raw log name of primitive
primitives = ["xor_256", "rng_256"] #TODO fill in with your own primitive log_type names matching index
 